Databricks notebook source
MAGIC %md
MAGIC Neste notebook, analisaremos os e-mails, e treinaremos o melhor modelo, para verificar se os emails referidos se tratam de SPAM

COMMAND ----------

In [ ]:
df = spark.read.csv('/FileStore/tables/classification.csv', header=True, inferSchema=True)

COMMAND ----------

In [ ]:
df.show()

COMMAND ----------

In [ ]:
# importar as bibliotecas que estaremos trabalhando
# Estamos selecionando 3 modelos de classificação, para definir, qual melhor modelo
import logging
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier , DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# Configura o logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

Definir as colunas features e destino(lable) para serem analisadas pelos modelos

In [ ]:
features_col =['word_count','has_attachment','is_reply']
target_col = 'is_spam'

In [ ]:
# Carregar os dados
logger.info("Carregando os dados..")
data = spark.read.format('csv').option('header',True).option('inferSchema',True).load('/FileStore/tables/classification.csv')

In [ ]:
# Convertendo as colunas
logger.info("Preparando os dados...") 
assembler = VectorAssembler(inputCols=features_col,outputCol='features')
data = assembler.transform(data).select('features',target_col)

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
# randomsplit define a divisão do arquivo de dados que vai ser treinada e que vai ser guardada
# seed controla a alteração ou não do arquivo, os numeros e pesos continuarão os mesmos
train_data, test_data = data.randomSplit([0.8, 0.2],seed=1234)

Treine os modelos utilizando os dados
treinando os 3 modelos de uma vez, armazenando tudo na variável "models"

In [ ]:
logger.info("Treinando os modelos...")

In [ ]:
models = {
  'Logistic_Regression': LogisticRegression(featuresCol='features',labelCol=target_col),
  'Random_Forest': RandomForestClassifier(featuresCol='features',labelCol=target_col),
  'Decision_Tree': DecisionTreeClassifier(featuresCol='features',labelCol=target_col),
}

Depois do treinamento dos modelos, temos que avaliar qual o melhor modelo utilizado na busca dos dados
Busca o melhor. na variável evaluator avaliando todas as saídas dos modelos e vai verificar qual vai ser a melhor
utilizando a métrica f1 scrore, que combina a precisão e record, a média harmônica ( essa métrica é bem utilizada em modelos que podem estar desbalanceados, ) deixar 45 a 50 % para cada uma das saídas
1 indica desempenho perfeito e 0 indica um desempenho muito ruim

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol=target_col,predictionCol="prediction",metricName="f1")
best_model = None
best_f1_score = 0

In [ ]:
for name, model in models.items():
    logger.info(f"Treinando e avaliando o modelo {name}... ")
    trained_model = model.fit(train_data)
    predictions = trained_model.transform(test_data)
    f1_score = evaluator.evaluate(predictions)
    logger.info(f"F1 Score do modelo {name}: {f1_score}")
    
    if f1_score > best_f1_score:
      best_f1_score = f1_score
      best_model = name

In [ ]:
logger.info(f"O Modelo com o melhor desempenho é {best_model} com F1 Score de {best_f1_score}.")